In [2]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)

        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

model = ResNet18(num_classes=10)
model.load_state_dict(torch.load("model.pth", weights_only=True, map_location=torch.device('cpu')))
model.eval()  # 切换到评估模式

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fa

In [6]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import torch
from torchvision import transforms
import pyperclip
import io
from PIL import Image, ImageTk, ImageGrab

map_word = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class PaintApp:
    def __init__(self, root):
        self.root = root
        self.root.title("GUI")

        self.main_frame = tk.Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        self.canvas = tk.Canvas(self.main_frame, bg="white", width=140, height=140)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.sidebar = tk.Frame(self.main_frame, width=200, bg="#f0f0f0")
        self.sidebar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label = tk.Label(self.sidebar, text="预测值为0", bg="#f0f0f0", font=("Arial", 14))
        self.label.pack(pady=20)

        self.eval_button = tk.Button(self.sidebar, text="推理", command=self.forward)
        self.eval_button.pack(pady=10)

        self.paste_button = tk.Button(self.sidebar, text="粘贴", command=self.paste)
        self.paste_button.pack(pady=10)

        self.clear_button = tk.Button(self.sidebar, text="清除", command=self.clear)
        self.clear_button.pack(pady=10)

        self.image = Image.new("RGB", (140, 140), "white")
        self.draw = ImageDraw.Draw(self.image)

    def get_clipboard_image(self):
        try:
            clipboard_data = pyperclip.paste()
            
            if clipboard_data.startswith("data:image/"):
                image_data = clipboard_data.split(",")[1]
                image_stream = io.BytesIO(base64.b64decode(image_data))
                image = Image.open(image_stream)
            else:
                image = ImageGrab.grabclipboard()
            
            if image is None:
                raise ValueError("剪切板中没有图像")
            
            return image
        except Exception as e:
            print(f"获取剪切板图像失败: {e}")
            return None

    def paste(self, event=None):
        self.image = self.get_clipboard_image()
        if self.image:
            photo_image = ImageTk.PhotoImage(self.image.resize((140, 140)))
            self.canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
            self.canvas.image = photo_image

    def forward(self, event=None):
        resized_image = self.image.resize((32, 32))
        
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0], std=[1])
        ])
        tensor_image = transform(resized_image)
        
        outputs = model(tensor_image.unsqueeze(0))
        print(outputs.data)
        _, predicted = torch.max(outputs.data, dim=1)
        self.label.config(text=f"预测值为{map_word[int(predicted)]}")
    def clear(self):
        self.canvas.delete("all")

        self.image = Image.new("RGB", (140, 140), "white")
        self.draw = ImageDraw.Draw(self.image)

        self.last_x, self.last_y = None, None

if __name__ == "__main__":
    root = tk.Tk()
    app = PaintApp(root)
    root.mainloop()


tensor([[-0.0489, -7.1481, -3.3034,  1.5893, -3.4346,  0.1859, -1.8412, -6.1383,
          6.0334, -4.4487]])
tensor([[-0.0489, -7.1481, -3.3034,  1.5893, -3.4346,  0.1859, -1.8412, -6.1383,
          6.0334, -4.4487]])
tensor([[ -7.2414, -14.1827,  -4.6415,   8.3984,  -2.7615,   9.0379,  -2.3469,
          -2.7946,  -6.9362,  -9.6751]])
tensor([[ -5.5202, -11.9724,  -5.8071,  10.1513,  -3.0799,   4.5606,  -4.0857,
          -0.9027,  -5.5029,  -7.5087]])
tensor([[ -6.7101, -16.4843,  -1.9581,   7.2758,  -3.8832,   7.7009,  -2.6581,
          -0.9913,  -6.6308, -10.2650]])
tensor([[ -6.0731, -13.7109,  -3.0840,   5.9586,  -0.5995,   4.8360,  -1.7917,
          -2.3065,  -2.2844,  -8.6470]])
tensor([[-0.4554, -0.3091, -1.6996,  2.6447, -4.7809, -3.7305,  1.7036, -5.5571,
         -4.1814, -4.4909]])
tensor([[-0.4554, -0.3091, -1.6996,  2.6447, -4.7809, -3.7305,  1.7036, -5.5571,
         -4.1814, -4.4909]])
tensor([[-2.8292,  8.3108, -8.6788,  1.0576, -7.5986, -3.0590, -4.0296, -3.4436,